# 4.文本分类

In [1]:
pip install datasets transformers sentence-transformers openai

In [2]:
import os
os.environ["HF_HOME"] = "/openbayes/home/huggingface"

## 4.1 加载数据

In [3]:
# datasets是hugging face提供的一个用于加载和处理各种公开数据集的库
from datasets import load_dataset

# 使用rotten_tomatoes数据集，这是文本情感分类任务中非常经典的数据集之一
data = load_dataset("rotten_tomatoes")
data

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [4]:
data["train"][0,-1]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 0]}

In [5]:
data["train"]["text"]

Column(['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .', 'effective but too-tepid biopic', 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .', "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one ."])

In [6]:
list(data["train"]["text"])

['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
 'effective but too-tepid biopic',
 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .',
 "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .",
 'the film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .',
 'offers that rare combination of entertainment and education .',
 'perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .',
 "steers turns i

## 4.2表征类模型
### 4.2.1使用和任务相关的模型

In [7]:
# 用一个已经训练好的情感分析模型，对 rotten_tomatoes 测试集做推理，并评估分类效果。
from transformers import pipeline

# 定义一个变量model_path，指定要加载的模型路径或模型名。
# 这里给的是HuggingFaceHub上模型的名字，pipeline会自动从HuggingFace下载模型和tokenizer。

model_path =  "cardiffnlp/twitter-roberta-base-sentiment-latest"

pipe = pipeline(
    model  =  model_path,
    tokenizer = model_path,
    return_all_scores = True # 默认情况下，pipeline只返回得分最高的标签，设置这个参数为True后，将返回所有分类标签的分数列表
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [8]:
# 小测试
pipe("I love this movie")


[[{'label': 'negative', 'score': 0.005434969440102577},
  {'label': 'neutral', 'score': 0.027071906253695488},
  {'label': 'positive', 'score': 0.9674930572509766}]]

In [9]:
data['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 1066
})

In [10]:
import numpy as np
from tqdm import tqdm # 用于在循环中显示进度条
from transformers.pipelines.pt_utils import KeyDataset # 导入KeyDataset,用于从字典数据中提取特定字段作为输入

y_pred = [] # 存放预测结果的列表
# 遍历数据集进行推理
# KeyDataset(data["test"], "text") 会从 data["test"] 中按顺序提取 "text" 字段作为输入
# tqdm 用于显示循环的进度条，total 指定数据总长度
for output  in tqdm(pipe(KeyDataset(data["test"],"text")), total=len(data['test'])):
    # pipe会自动使用模型内置的label分类
    # 取负面情感的得分
    negative_score = output[0]["score"]

    # 取正面情感的得分
    positive_score = output[2]["score"]    
    # 根据负面和正面得分，判断哪个得分更高
    # np.argmax 返回数组中最大值的索引：
    #   如果负面得分大 → 返回 0
    #   如果正面得分大 → 返回 1
    assignment = np.argmax([negative_score,positive_score])
    # 将预测结果保存到列表
    # 这里 assignment 是 0 或 1，表示二分类结果（负/正）
    y_pred.append(assignment)

100%|██████████| 1066/1066 [02:10<00:00,  8.15it/s]


In [11]:
from sklearn.metrics import classification_report # 导入sklearn的分类评估函数，用于生成分类报告
def evaluate_performance(y_true,y_pred):
  """
  评估分类模型性能的函数
  参数：
      y_true: list 或 ndarray
          真实标签（ground truth），通常是 0/1 的数组或列表
      y_pred: list 或 ndarray
          模型预测的标签，与 y_true 对应
  功能：
      生成并打印分类任务的性能报告，包括：
      - precision（精确率）
      - recall（召回率）
      - f1-score（F1 分数）
      - support（每个类别的样本数）
  """
  performance = classification_report(
    y_true,# 真实标签
    y_pred,#预测标签
    target_names=["Negative Review", "Positive Review"]  # 指定标签的可读名称
  )
  print(performance)  # 打印性能报告

In [12]:
evaluate_performance(list(data["test"]["label"]), y_pred)

                 precision    recall  f1-score   support

Negative Review       0.76      0.88      0.81       533
Positive Review       0.86      0.72      0.78       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066



### 利用文本向量做分类
监督学习（训练分类模型）

In [13]:
# 先把文本转成语义向量，再用逻辑回归在向量空间里做情感二分类，并评估效果


from sentence_transformers import SentenceTransformer

# 从hugging face hub加载一个预训练模型
# sentence-transformers/all-distilroberta-v1基于DistilRoBERTa 架构，并在各种文本相似度任务上进行了微调
# 这个模型能够将输入的文本转换成密集向量（即嵌入），这些向量能捕捉文本的语义信息
model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

# 将训练集中的文本数据，转换为嵌入向量
# model.encode()方法负责执行转换
# show_progress_bar = True参数会在转换过程中显示一个进度条，方便跟踪进度
train_embeddings =model.encode(list(data["train"]["text"]), show_progress_bar = True)
test_embeddings =model.encode(list(data["test"]["text"]), show_progress_bar = True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/267 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

In [14]:
from sklearn.linear_model import LogisticRegression

# 训练一个逻辑回归模型对我们的训练集嵌入进行分类
# 逻辑回归是一种广义线性模型，常用于二分类或多分类任务
# 在这里，它将学习如何将文本嵌入向量映射到对应的标签

# 实例化一个LogisticRegression模型对象，并通过参数来调优模型的性能
clf = LogisticRegression(
    random_state = 42,# 设置随机种子，保证每次运行结果的可复现性
    max_iter=1000,# 增加最大迭代次数，以确保模型在收敛之前有足够的训练机会
)

# 使用训练集数据拟合（训练）模型。
# train_embeddings: 训练集的嵌入向量，是模型的输入特征。
# data["train"]["label"]: 训练集的真实标签，是模型的学习目标。
clf.fit(train_embeddings, list(data["train"]["label"]))

# 使用训练好的模型对测试集的嵌入向量进行预测
# 这步是将模型应用到未见过的新数据上，以评估其泛化能力
y_pred = clf.predict(test_embeddings)

# 调用之前定义的evaluate_performance 来评估模型的性能。
evaluate_performance(list(data["test"]["label"]), y_pred)

                 precision    recall  f1-score   support

Negative Review       0.79      0.82      0.80       533
Positive Review       0.81      0.78      0.79       533

       accuracy                           0.80      1066
      macro avg       0.80      0.80      0.80      1066
   weighted avg       0.80      0.80      0.80      1066



Zero-shot方式分类

In [ ]:
import numpy  as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity

# ----------------- 计算平均嵌入向量 -----------------
# 将训练集的嵌入向量 (train_embeddings) 和对应的标签 (data["train"]["label"]) 合并成一个 Pandas DataFrame。
df = pd.DataFrame(np.hstack([train_embeddings, np.array(list(data["train"]["label"])).reshape(-1, 1)]))
print(df.shape, train_embeddings.shape) # 打印合并后的 DataFrame 形状和原始嵌入向量的形状(文本数，向量维度)

# 按标签对嵌入向量进行分组，并计算每个标签的平均嵌入向量。768 是 label 所在的列索引
averaged_target_embeddings = df.groupby(768).mean().values
print(averaged_target_embeddings.shape) # 打印平均嵌入向量的形状

# ----------------- 寻找最佳匹配并进行预测 -----------------
# 计算测试集嵌入向量 (test_embeddings) 和平均目标嵌入向量 (averaged_target_embeddings) 之间的余弦相似度。
# 余弦相似度用于衡量两个向量方向上的相似性，值介于 -1 和 1 之间。
# 结果 sim_matrix 的形状为 (测试样本数, 类别数)，其中 sim_matrix[i][j] 表示第 i 个测试样本与第 j 个类别的平均嵌入向量的相似度。
sim_matrix = cosine_similarity(test_embeddings, averaged_target_embeddings)
print(sim_matrix.shape)

# 找到每个测试样本最相似的类别。
# np.argmax(sim_matrix, axis=1) 返回每行（axis=1）最大值的索引。
# 这个索引就代表了预测的类别标签。
y_pred = np.argmax(sim_matrix, axis=1)

# ----------------- 评估模型性能 -----------------
# 比较测试集的真实标签 (data["test"]["label"]) 和我们基于余弦相似度得到的预测标签 (y_pred)。
# 这个评估函数通常会输出分类报告，包含准确率、精确率、召回率和 F1 分数等指标。
evaluate_performance(data["test"]["label"], y_pred)

(8530, 769) (8530, 768)
(2, 768)
(1066, 2)
                 precision    recall  f1-score   support

Negative Review       0.74      0.79      0.76       533
Positive Review       0.77      0.73      0.75       533

       accuracy                           0.76      1066
      macro avg       0.76      0.76      0.76      1066
   weighted avg       0.76      0.76      0.76      1066



In [ ]:
# 用另外一种方式获取 label 的 embedding
label_embeddings = model.encode(["A negative review",  "A positive review"])

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

# Find the best matching label for each document
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)


evaluate_performance(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.75      0.61      0.68       533
Positive Review       0.67      0.80      0.73       533

       accuracy                           0.71      1066
      macro avg       0.71      0.71      0.70      1066
   weighted avg       0.71      0.71      0.70      1066



## 4.3生成模型做分类
#### 4.3.1 Encoder-docer模型

In [26]:
# 创建一个text2text-generation类型的管道
pipe = pipeline(
    "text2text-generation",
    model = "google/flan-t5-small",)

# 定义一个通用的prompt，用于引导模型进行情感判断
prompt = "Is the following sentence positive or negative ?"
#使用 map 方法对每一行数据执行同样的操作：
#- 取出原始文本（假设字段名为 'text'）
#- 将 prompt 与文本拼接起来
#- 将结果存入一个新的键 't5' 中（这个字段将来复用于模型输入）
data = data.map(lambda example:{"t5": prompt + example['text']})
data

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

Device set to use cpu


Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
})

In [27]:
# 推理
# 创建一个空列表，用来保存最终的结果
y_pred = []

# KeyDataset(data['test'], "t5") -> 只取 data['test'] 数据集中字段名为 't5' 的输入，逐条喂给模型
for output in tqdm(pipe(KeyDataset(data['test'],"t5")),total=len(data['test'])):
  # output 是列表，每个元素是模型输出的字典，例如：
  # output = [{"generated_text": "negative"}]
  text = output[0]["generated_text"]
  y_pred.append(0 if text == "negative" else 1)

100%|██████████| 1066/1066 [02:59<00:00,  5.93it/s]


In [28]:
print(y_pred)

[1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 

In [29]:
evaluate_performance(data["test"]["label"], y_pred)


                 precision    recall  f1-score   support

Negative Review       0.82      0.87      0.84       533
Positive Review       0.86      0.80      0.83       533

       accuracy                           0.84      1066
      macro avg       0.84      0.84      0.84      1066
   weighted avg       0.84      0.84      0.84      1066



#### 4.3.2 Decoder模型

In [30]:
pipe = pipeline(
    "text-generation",
    model = "gpt2"
)

prompt = "Is the following sentence positive or negative ?"
data = data.map(lambda example: {"gpt2": prompt + example['text']})
data

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Device set to use cpu


Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5', 'gpt2'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 't5', 'gpt2'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 't5', 'gpt2'],
        num_rows: 1066
    })
})

In [ ]:
# 推理
y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "gpt2")), total=len(data["test"])):
    text = output[0]["generated_text"]
    y_pred.append(0 if text == "negative" else 1)

evaluate_performance(data["test"]["label"], y_pred)

  1%|          | 8/1066 [01:35<3:41:22, 12.55s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
